In [23]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import argparse
import torch
#from callbacks import CSVParamLogger
from scipy.stats import spearmanr, rankdata
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import RandomSampler
from tqdm import tqdm
from transformers import *
#from utils import torch_to_numpy
#from model_dataset import QuestDataset
#from apex import amp
#from apex.parallel import DistributedDataParallel as DDP
#from apex.optimizers import FusedAdam
import logging
from model_dataset import QuestDataset
from utils.loss_function import *
from utils.metric import *
from utils.lrs_scheduler import *
from utils.file import *
from pathlib import Path

In [5]:
PATH_TO_CKPT_CONFIG = Path(os.getcwd() + '/input/data/')

In [239]:
tokenizer = BertTokenizer(str(PATH_TO_CKPT_CONFIG / "vocab.txt"), do_basic_tokenize=True, do_lower_case=False)
orig_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

I0503 18:23:21.765520  6988 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\admin\.cache\torch\transformers\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [21]:
len(tokenizer)
len(orig_tokenizer)

28996

In [15]:
v1 = orig_tokenizer.vocab.keys()
v2 = tokenizer.vocab.keys()
len(v1)
len(v2)

110000

In [65]:
a=list(set(v2)-set(v1))
a[:10]
len(a)

81004

In [25]:
#orig_bert = BertForPreTraining.from_pretrained("bert-base-cased")

In [26]:
state_dict = orig_bert.state_dict()

In [32]:
orig_bert.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [104]:
input_ids = torch.tensor([[1,2,3,4]])
token_type_ids = torch.tensor([[0,0,0,0]])
attention_mask = torch.tensor([[1,1,1,1]])
outputs = orig_bert.bert(input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask)

In [98]:
len(outputs)
print(outputs[1].shape)
print(outputs[0].shape)


torch.Size([1, 768])
torch.Size([1, 4, 768])


In [159]:
[n for n,p in orig_bert.named_parameters()]

['bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias',
 'bert.encoder.layer.0.attention.self.query.weight',
 'bert.encoder.layer.0.attention.self.query.bias',
 'bert.encoder.layer.0.attention.self.key.weight',
 'bert.encoder.layer.0.attention.self.key.bias',
 'bert.encoder.layer.0.attention.self.value.weight',
 'bert.encoder.layer.0.attention.self.value.bias',
 'bert.encoder.layer.0.attention.output.dense.weight',
 'bert.encoder.layer.0.attention.output.dense.bias',
 'bert.encoder.layer.0.attention.output.LayerNorm.weight',
 'bert.encoder.layer.0.attention.output.LayerNorm.bias',
 'bert.encoder.layer.0.intermediate.dense.weight',
 'bert.encoder.layer.0.intermediate.dense.bias',
 'bert.encoder.layer.0.output.dense.weight',
 'bert.encoder.layer.0.output.dense.bias',
 'bert.encoder.layer.0.output.LayerNorm.weight',
 'bert.encoder.layer

In [99]:
prediction_scores, seq_relationship_score = orig_bert.cls(outputs[0],outputs[1])
print(seq_relationship_score.shape)
print(prediction_scores.shape)

torch.Size([1, 2])
torch.Size([1, 4, 28996])


In [100]:
outputs = orig_bert(input_ids,token_type_ids=token_type_ids,attention_mask=attention_mask)
len(outputs)
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 4, 28996])
torch.Size([1, 2])


In [50]:
state_dict = orig_bert.state_dict()
for key in state_dict.keys():
    if 'cls' in key:
        print(key, "\t\t\t", state_dict[key].shape)
    

cls.predictions.bias 			 torch.Size([28996])
cls.predictions.transform.dense.weight 			 torch.Size([768, 768])
cls.predictions.transform.dense.bias 			 torch.Size([768])
cls.predictions.transform.LayerNorm.weight 			 torch.Size([768])
cls.predictions.transform.LayerNorm.bias 			 torch.Size([768])
cls.predictions.decoder.weight 			 torch.Size([28996, 768])
cls.predictions.decoder.bias 			 torch.Size([28996])
cls.seq_relationship.weight 			 torch.Size([2, 768])
cls.seq_relationship.bias 			 torch.Size([2])


In [66]:
new_tokens_as_orig_indices = [[i] for i in range(len(orig_tokenizer.vocab))] + [orig_tokenizer.encode(t, add_special_tokens=False) for t in a]

In [75]:
#x=[[i] for i in range(len(orig_tokenizer.vocab))]
new_tokens_as_orig_indices[28996]

[2373, 3263, 168, 22605]

In [94]:
orig_embedding[[1,2,3]].shape

torch.Size([3, 768])

In [80]:
orig_embedding = state_dict["bert.embeddings.word_embeddings.weight"]
orig_embedding.shape[-1]
len(1)>0

TypeError: object of type 'int' has no len()

In [109]:
import torch.nn as nn
classifier = nn.Linear(768, 5)
dropout = nn.Dropout(0.1)
sequence_output, pooled_output = outputs[:2]
#mean_pooled_output = torch.mean(sequence_output, dim=1)
#mean_pooled_output = dropout(mean_pooled_output)
#logits = classifier(mean_pooled_output)
logits = classifier(dropout(torch.mean(sequence_output,dim=1)))
pred_scores, seq_rel_score = orig_bert.cls(sequence_output, pooled_output)

In [108]:
logits.shape

torch.Size([1, 5])

In [172]:
#c = torch.nn.BCELoss()(np.array([0.8]),np.array([1.0]))
a=torch.tensor([0.8])
a.size()

torch.Size([1])

In [139]:
logits = torch.tensor([[[1.0,2.1,4.2,0.5,1.1],[1.1,2.2,4.3,0.6,1.2],[1.2,2.3,4.4,0.7,1.3]]]) 
print(mlm_inp.shape)
targets = torch.tensor([[-1,44,-1]])
print(targets.shape)
n_samples = np.prod(targets.shape)
print(n_samples)
print(logits.view(n_samples,-1))
print(targets.view(n_samples))


torch.Size([1, 3, 5])
torch.Size([1, 3])
3
tensor([[1.0000, 2.1000, 4.2000, 0.5000, 1.1000],
        [1.1000, 2.2000, 4.3000, 0.6000, 1.2000],
        [1.2000, 2.3000, 4.4000, 0.7000, 1.3000]])
tensor([-1, 44, -1])


In [155]:
logits = torch.tensor([[0.1,0.4],[1.1,0.2],[1,0]])
print(logits)
b=torch.argmax(logits.view(-1, 2), dim=-1)
#logits.view(-1,2)
print(b)
b.view(-1)
logits.view(-1)

tensor([[0.1000, 0.4000],
        [1.1000, 0.2000],
        [1.0000, 0.0000]])
tensor([1, 0, 0])


tensor([0.1000, 0.4000, 1.1000, 0.2000, 1.0000, 0.0000])

In [160]:
a = torch.tensor([[0.1,0.4],[1.1,0.2],[1,0]])
a.shape
b = torch.tensor([[0.1,0.4],[1.1,0.2],[1,0]])

np.stack(1a,b)

torch.Size([3, 2])

In [186]:
47 %2

1

In [188]:
a=torch.tensor([[1,2,3]])
b=torch.tensor([[1,2,3],[4,5,6]])
c=torch.cat((a,b))

c.shape

torch.Size([3, 3])

In [189]:
    class MLMloss(nn.CrossEntropyLoss):
        def forward(self,logits,targets):
            n_samples = np.prod(targets.shape)
            loss = super(MLMloss,self).forward(logits.view(n_samples,-1),targets.view(n_samples))
            return loss
    class SOPloss(nn.CrossEntropyLoss):
        def forward(self,logits,targets):
            n_samples = np.prod(targets.shape)
            loss = super(SOPloss,self).forward(logits.view(-1,2),targets.view(-1))
            return loss

    class PretrainingLoss(torch.nn.Module):
        def __init__(self, targets_alpha=1.0):
            super(PretrainingLoss, self).__init__()
            self.mlm_loss = MLMloss(ignore_index=-1)
            self.sop_loss = SOPloss()
            self.bce = torch.nn.BCEWithLogitsLoss()
            self.targets_alpha = targets_alpha

        def forward(self, logits, targets):
            return (
                    self.mlm_loss(logits[0], targets[0])
                    + self.sop_loss(logits[1], targets[1])
                    + self.targets_alpha * self.bce(logits[2], targets[2])
            )

    class MLMPerplexity(MLMloss):
        __name__ = "mlm_perplexity"
        def forward(self, logits, targets):
            logits, targets = logits[0], targets[0]
            loss = super(MLMPerplexity, self).forward(logits, targets)
            perplexity = 2 ** loss
            return float(perplexity)

    def sop_accuracy(logits, targets):
        logits, targets = logits[1], targets[1]
        pred = torch.argmax(logits.view(-1, 2), dim=-1)
        targets = targets.view(-1)
        return float(torch.mean((pred == targets).float()))

In [298]:
dummy_logits = (  torch.tensor([[[5.9,1.2,1.3],[0.6,1.5,0.1]]],dtype=torch.float) ,torch.tensor([[0.1,0.9]]) ,torch.tensor([[0.1,0.2,0.3,0.4,0.5]]) )
dummy_targets = (  torch.tensor([[-1,2]],dtype=torch.long) ,torch.tensor([[0]]) ,torch.tensor([[0.1,0.2,0.3,0.4,0.5]]) )

In [259]:
dummy_logits[1].shape

torch.Size([1, 2])

In [300]:
#a = MLMloss(ignore_index=-1)(dummy_logits[0], dummy_targets[0])
#a.item()==0.9608293771743774

#b=SOPloss()(dummy_logits[1], dummy_targets[1])
#b.item()==1.1711006164550781
#c=sop_accuracy(dummy_logits, dummy_targets)
#c==0

#if (a.item()==0.9608293771743774) & (b.item()==1.1711006164550781) & (c==0):
#    print("success")
MLMPerplexity(ignore_index=-1)(dummy_logits,dummy_targets)

3.7391037940979004

In [238]:
TARGETS = ['question_score','question_views','question_favs','answer_score','is_answer_accepted']
class QuestMLMDataset(QuestDataset):
    def __init__(self, df, tokenizer, max_len=512, content = "Question_Answer", mlm_prob = 0.15,
                 non_masked_idx=-1,sop_prob=0.5,target_cols = TARGETS):
        super(QuestMLMDataset,self).__init__(df = df, target_cols=target_cols, max_len=max_len, content=content,tokenizer=tokenizer)
        self.mlm_probability = mlm_prob
        self.sop_prob = sop_prob
        self.non_masked_idx = non_masked_idx
        self.tokenizer = tokenizer
        self.cls_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.cls_token)
        self.sep_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.sep_token)
        self.mask_token_idx = tokenizer.convert_tokens_to_ids(tokenizer.mask_token)

    def _mask_tokens(self,inputs,masked_random_replace_prob=0.2):
        labels = inputs.clone()
        masked_indices = torch.bernoulli(torch.full(labels.shape,self.mlm_probability)).bool()
        for special_tokens in [self.cls_token_idx,self.sep_token_idx,self.mask_token_idx]:
            masked_indices &= inputs != special_tokens
        labels[~masked_indices] = self.non_masked_idx
        indices_replaced = (torch.bernoulli(torch.full(labels.shape,1 - masked_random_replace_prob)).bool() & masked_indices)
        inputs[indices_replaced] = self.mask_token_idx
        indices_random = (torch.bernoulli(torch.full(labels.shape,0.5)).bool() & masked_indices & ~indices_replaced)
        random_words = torch.randint(len(self.tokenizer),labels.shape,dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]
        return inputs,labels

    def __getitem__(self, index):
        title, body, answer = self._get_text(index)
        numeric_targets = self.get_label(index)
        sop_label=0
        if np.random.uniform(0,1) < self.sop_prob:
            sop_label=1
            perm = list(np.random.permutation(range(3)))
            if perm == [0,1,2]:
                perm = [0,2,1]
            title, body, answer = [[title, body, answer][i] for i in perm]
        input_ids, token_type_ids, attention_mask  = self.get_token_ids(title, body, answer)
        input_ids, token_type_ids, attention_mask = map(torch.LongTensor,[input_ids, token_type_ids, attention_mask])
        input_ids, labels = self._mask_tokens(torch.LongTensor(input_ids))
        return ((input_ids, token_type_ids, attention_mask),(labels, sop_label, numeric_targets))

In [241]:
stackx_data = pd.read_csv( "./input/qa_stackexchange_cleaned.csv", nrows=5)
train_dataset = QuestMLMDataset(stackx_data, tokenizer, target_cols=TARGETS)

In [243]:
inputs,targets = train_dataset.__getitem__(1)

In [278]:
a,b,c = targets
a.shape

torch.Size([512])

In [279]:
from transformers import AdamW, BertTokenizer, BertForPreTraining, get_linear_schedule_with_warmup
class BertPretrain(BertForPreTraining):
    def __init__(self,config, num_labels):
        super(BertPretrain,self).__init__(config,)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
    def forward(self, input_ids=None,token_type_ids=None,attention_mask=None):
        outputs = self.bert(input_ids, token_type_ids, attention_mask)
        sequence_output, pooled_output = outputs[:2]
        logits = self.classifier(self.dropout(torch.mean(sequence_output,dim=1)))
        pred_scores, seq_rel_score = self.cls(sequence_output, pooled_output)
        outputs = (pred_scores,seq_rel_score,logits)
        return outputs


In [285]:
config = BertConfig.from_json_file("./input/data/config.json")
config.vocab_size

110000

In [286]:
model = BertPretrain(config, 5)

In [288]:
[n for n, p in model.named_parameters()]

['bert.embeddings.word_embeddings.weight',
 'bert.embeddings.position_embeddings.weight',
 'bert.embeddings.token_type_embeddings.weight',
 'bert.embeddings.LayerNorm.weight',
 'bert.embeddings.LayerNorm.bias',
 'bert.encoder.layer.0.attention.self.query.weight',
 'bert.encoder.layer.0.attention.self.query.bias',
 'bert.encoder.layer.0.attention.self.key.weight',
 'bert.encoder.layer.0.attention.self.key.bias',
 'bert.encoder.layer.0.attention.self.value.weight',
 'bert.encoder.layer.0.attention.self.value.bias',
 'bert.encoder.layer.0.attention.output.dense.weight',
 'bert.encoder.layer.0.attention.output.dense.bias',
 'bert.encoder.layer.0.attention.output.LayerNorm.weight',
 'bert.encoder.layer.0.attention.output.LayerNorm.bias',
 'bert.encoder.layer.0.intermediate.dense.weight',
 'bert.encoder.layer.0.intermediate.dense.bias',
 'bert.encoder.layer.0.output.dense.weight',
 'bert.encoder.layer.0.output.dense.bias',
 'bert.encoder.layer.0.output.LayerNorm.weight',
 'bert.encoder.layer

In [294]:
train_loader = DataLoader(train_dataset,batch_size=4)

In [295]:
len(train_loader)

2

In [297]:
for tr_batch_i, (inputs,targets) in enumerate(train_loader):
    input_ids, token_type_ids, attention_mask = inputs
    targets_mlm, targets_sop, targets_extra = targets
    print("input_ids",input_ids.shape)
    print("token_type_ids",token_type_ids.shape)
    print("attention_mask",attention_mask.shape)
    print("targets_mlm",targets_mlm.shape)
    print("targets_sop",targets_sop.shape)
    print("targets_extra",targets_extra.shape)
    logits = model(input_ids, token_type_ids, attention_mask)
    logits_mlm, logits_sop, logits_extra = logits
    print("logits_mlm",logits_mlm.shape)
    print("logits_sop",logits_sop.shape)
    print("logits_extra",logits_extra.shape)
    
    

input_ids torch.Size([4, 512])
token_type_ids torch.Size([4, 512])
attention_mask torch.Size([4, 512])
targets_mlm torch.Size([4, 512])
targets_sop torch.Size([4])
targets_extra torch.Size([4, 5])
logits_mlm torch.Size([4, 512, 110000])
logits_sop torch.Size([4, 2])
logits_extra torch.Size([4, 5])
input_ids torch.Size([1, 512])
token_type_ids torch.Size([1, 512])
attention_mask torch.Size([1, 512])
targets_mlm torch.Size([1, 512])
targets_sop torch.Size([1])
targets_extra torch.Size([1, 5])
logits_mlm torch.Size([1, 512, 110000])
logits_sop torch.Size([1, 2])
logits_extra torch.Size([1, 5])
